In [2]:
# Hidden cell for imports
import pychor
import galois

# Defining & Proving Security

This chapter discusses how to prove that a protocol is secure. We start with the definition of a threat model, which is a prerequisite for proving security. Then, we describe the most common proof technique in MPC: simulator proofs in the real/ideal paradigm.

## Threat Models

A *threat model* describes the assumptions we make about the behavior of parties participating in a secure protocol.

When we argue (or prove) that a protocol is secure, we do so with respect to a particular threat model. The idea of security makes sense only in the context of a threat model, as we'll see in this chapter, so a secure protocol should always come with a complete description of the intended threat model.

### The Adversary and Corruption

The *adversary* is a conceptual abstraction of what we're trying to defend against. "The adversary" is typically thought of as a single entity who may *corrupt* some of the parties participating in the protocol. Corrupted parties reveal everything they know to the adversary (and in some cases, follow the adversary's commands). The adversary can learn or compute anything that any of the corrupted parties could learn or compute on their own, and the notion of collusion between parties is captured by the idea that multiple parties may be corrupted simultaneously and the adversary learns all of their combined knowledge.

Parties that are not corrupted are called *honest*. We typically assume that honest parties do not share any information with any other party, except as directed by the protocol.

### Semi-Honest and Malicious Security

Honest parties always follow the protocol. But how do corrupted parties behave?

In the *semi-honest* (also called *passive*) model, corrupted parties *also* follow the protocol. In this model, *all* parties follow the protocol.

In the *malicious* (also called *active*) model, corrupted parties *may deviate* from the protocol, as instructed by the adversary. In this model, honest parties still follow the protocol, but corrupted ones don't.

Malicious security is clearly the stronger threat model - it makes fewer assumptions about the behavior of corrupted parties. Which one is "correct"? There is no single right answer - it depends on what expectations you can safely make about how parties running the protocol will behave in real life. In particular, is it realistic to believe that corrupted parties will still follow the protocol? In most cases, probably not - so semi-honest security is typically considered a fairly weak threat model.

Why consider semi-honest security at all, then? It turns out that building protocols with semi-honest security is often much easier than building malicious-secure protocols, and semi-honest protocols can often be a useful stepping-stone to more complicated malicious-secure ones. We'll see examples of this in later chapters.

### Honest and Dishonest Majorities

Besides semi-honest and malicious, the other major consideration in a threat model is: how many parties may be corrupted?

In the *honest majority* setting, we assume that a majority of the parties are honest (not corrupted).

In the *dishonest majority* setting (also called *$n-1$ security*) we assume that $n-1$ out of the $n$ parties may be corrupted - in other words, everyone except a single person may be controlled by the adversary!

The dishonest majority assumption is the strongest one we can make. If $n$ parties are corrupted, then security is impossible (or maybe vacuous?) - the adversary knows everything all $n$ parties know, including their secrets.

The dishonest majority assumption is clearly stronger than the honest majority assumption, but assuming an honest majority often makes it much easier to construct secure protocols - we'll see several examples of protocols in later chapters. In real life, an honest majority might be a very reasonable assumption - in many cases, secure protocols are designed to protect against rare "bad actors" rather than situations where most participants are behaving badly.

In protocols that involve different classes of parties (e.g. clients and servers), it's common to state the assumptions precisely in terms of those classes, and it's possible to mix the two (e.g. we assume an honest majority of clients, but a dishonest majority of servers).

It's also common to be more precise about how large the honest majority is. Many secure protocols for [federated learning](https://en.wikipedia.org/wiki/Federated_learning), for example, assume that an even smaller fraction of parties may be corrupted (e.g. as few as 5% or 10%). For protocols involving hundreds or even thousands of parties, this may be a very reasonable assumption - it's generally more difficult for an adversary to corrupt a large number of parties in real life.

### How Many Participants?

WIP (2pc vs 3pc vs mpc, do more parties make it better?)

## Defining Security: The Real/Ideal Paradigm

The most common way to define security for MPC protocols is via the *real/ideal paradigm*. The idea is to define two artifacts:

1. The *ideal functionality*, which computes an output with the aid of a *trusted third party*
2. The *protocol*, which computes an output without the use of a trusted third party

Even though it's not called a protocol, the ideal functionality is usually written like one: it's specified in terms of a set of parties and the messages they send each other. The difference is that the ideal functionality includes an imaginary party who doesn't exist in the real world: a trusted third party, assumed to be honest and trusted by all of the other parties.

Security of the protocol is defined by relating the ideal functionality and the protocol. We say that a protocol *securely realizes* an ideal functionality if:

1. It's correct: the outputs of the ideal functionality and protocol are identical
2. It reveals no additional information: the adversary does not learn anything extra by observing execution of the protocol, compared to what they learn by observing execution of the ideal functionality

The ideal functionality serves as the *specification* of what security means for the protocol we're defining - it defines what information is allowed to leak out to the adversary.

Why do we define security this way? Why can't we just say that the protocol reveals *nothing* to the adversary? A definition like that would actually be *too* strong: it would prevent us from writing useful protocols, because most protocols reveal some information *on purpose*, via the protocol's output. Using the ideal functionality as a specification of allowed leakage makes it possible to define useful protocols while precisely limiting what the adversary can learn.

````{admonition} Further reading: Real/Ideal Paradigm
:class: seealso

For more information on the real/ideal paradigm, see **Section 2.3 of [Pragmatic MPC](https://securecomputation.org).**
````

### The Ideal World and Ideal Functionality

The ideal functionality is essentially the protocol we would write in the *ideal world*: a world that has a trusted third party who is honest (not corrupted by the adversary) and whom everybody trusts. The ideal world doesn't exist - if we had a trusted third party, we wouldn't need secure protocols in the first place!

An ideal functionality looks like a protocol, but involves a special party (the trusted third party) whose name usually matches the name of the functionality you're defining. For example, our summation protocol might have an ideal functionality called $\mathcal{F}_\text{sum}$, and the trusted third party in its definition would also be called $\mathcal{F}_\text{sum}$. Figures 2.1, 2.2, and 2.3 in [Pragmatic MPC](https://securecomputation.org) define some commonly-used ideal functionalities using this kind of notation.

We'll define ideal functionalities by writing them in the same style as protocols, but we'll use an additional party whose name matches the name of the functionality. Ideal functionalities will generally (but not always) follow a simple structure:

1. The parties send their inputs to the functionality (the trusted third party)
2. The functionality computes the outputs locally
3. The functionality sends the outputs to the appropriate parties

Since one party (the functionality) is allowed to know all the inputs at once, the ideal functionality is usually simple and easy to define. Here's an ideal functionality for our summation task:

In [8]:
GF = galois.GF(97)

p1 = pychor.Party('p1')
p2 = pychor.Party('p2')
Fsum = pychor.Party('Fsum')

def functionality_sum(x, y):
    x.send(p1, Fsum)
    y.send(p2, Fsum)
    total = x + y
    total.send(Fsum, p1)
    total.send(Fsum, p2)
    return total

Since we've implemented it, we can run the functionality just like a regular protocol. But it's not common to do this - the ideal functionality's purpose is to serve as a specification rather than an implementation.

### The Real World

In the *real world*, no trusted third party is available, so we run the protocol we've written that doesn't require one. Here's a copy of the summation protocol. To prove the protocol is secure, we want to prove that it doesn't reveal any more information to the adversary than the ideal functionality does.

In [9]:
def protocol_sum(x, y):
    x1, x2 = share(x, 2).unlist(2)
    y1, y2 = share(y, 2).unlist(2)
    x2.send(p1, p2)
    y1.send(p2, p1)
    sum1 = x1 + y1
    sum2 = x2 + y2
    sum1.send(p1, p2)
    sum2.send(p2, p1)
    total = reconstruct([sum1, sum2])
    return total

## The Simulator and Simulator Security

How do we prove that the adversary doesn't learn more in the real world (by observing the protocol) than they do in the ideal world (by observing the functionality)?

For most MPC protocols, proving security in the presence of a *semi-honest adversary* is done by showing the existence of a *simulator* (usually by actually constructing it). The simulator is a program that simulates what the adversary would have seen during an execution of the protocol, without actually running the protocol, and *without access to the inputs of the honest parties*. The proof is successful if it's impossible to distinguish between observations of the simulator and observations of the real protocol.

### Proving Correctness

Before we can prove security, we need to prove that the protocol is correct: it computes the same thing as the ideal functionality. This part of the proof is usually easier than proving security, since protocols are generally constructed to make it easy to see correctness - and correctness of the protocol is usually pretty easy to test just by running it.

In our summation example, the ideal functionality clearly computes the result $total = x + y$ and returns `total` to both parties. In the protocol, both parties compute:

$$
\begin{align*}
total &= sum1 + sum2 \tag{definition of the protocol}\\
&= x1 + y1 + x2 + y2 \tag{definition of the protocol}\\
&= (x1 + x2) + (y1 + y2) \tag{algebra}\\
&= x + y \tag{definition of secret sharing}\\
\end{align*}
$$

So, we know that `functionality_sum(x, y) = protocol_sum(x, y)`.

### Proving Semi-Honest Security with a Simulator

The idea behind simulator proofs is to show that:
- Anything the adversary can learn by observing the execution of the protocol
- They *could have learned* by observing the execution of the ideal functionality

If we can prove this, then it means *the protocol reveals no more information than the ideal functionality*, which is our definition of security.

How do we prove this? We do it by building a *simulator*: a program that represents a constructive proof of the above bullet points. In particular, the simulator must:
- Compute everything the adversary can observe during execution of the protocol
- With access only to the results of the ideal functionality, and *without* access to the inputs of the honest parties

This might sound tricky: how do we compute everything the adversary observes, if we're missing some of the input? The property that makes it possible is that the simulator *does* have access to the *output* of the ideal functionality, and most simulator proofs use the output to "work backwards" to the other values the adversary observes during the protocol.

To build a correct simulator, we'll use a structured form of *hybrid argument*. The simulator must compute values for the things observed by the adversary such that the joint probability distribution over these values is identical between the simulator and real protocol. Following a structured approach helps ensure we don't make a mistake.

A *hybrid argument* involves starting from the original protocol (using all of the inputs), and then modifying the program step by step until we arrive at the simulator (using only the corrupt party's inputs). Each step in the chain is called a *hybrid*. For more information on the hybrid technique, see [Section 2.4 of *The Joy of Cryptography*](https://joyofcryptography.com/provsec/#sec.prove).

Constructing the simulator only proves *semi-honest security*, because we assume that the parties correctly execute the protocol. Proving malicious security is related but more complicated; see [Section 2.3.3 of Pragmatic MPC](https://securecomputation.org) for details.

When we build a simulator, we need to decide which parties are corrupted. To ensure security for all parties, we need to build separate simulators for each possible subset of corrupted parties, or a general simulator that works for any of these subsets. In many cases, we can assume a particular party is corrupted, build the simulator, and then argue that the same idea generalizes to other parties. For now, to simplify the proof, we'll assume that *P1 is corrupted* and *P2 is honest*.

For our summation example, we'll start with `protocol_sum`, but extended with one extra argument: the output of the ideal functionality. The adversary will clearly observe the same values as the real protocol in this hybrid, since it's identical to the real protocol.

In [ ]:
def hyb1_sum(x, y, functionality_output):
    x1, x2 = share(x, 2).unlist(2)
    y1, y2 = share(y, 2).unlist(2)
    x2.send(p1, p2)
    y1.send(p2, p1)
    sum1 = x1 + y1
    sum2 = x2 + y2
    sum1.send(p1, p2)
    sum2.send(p2, p1)
    total = reconstruct([sum1, sum2])
    return total

The structure of our proof will involve two kinds of steps:

1. We'll maintain a set of *constraints*: mathematical equations we know are true. We'll introduce and manipulate constraints using *constraint moves*, described next.
2. We'll build new hybrids in the sequence using *hybrid moves*, described after that.

The last hybrid in the sequence will be the simulator.

#### Constraint Moves

During our proof, we'll need to keep track of facts we know are true and manipulate them to come up with new facts. To do this, we'll introduce *constraints* and use them in the proof. A constraint is just an equation involving some of the program variables. The following *constraint moves* are valid rules of thumb you can use to introduce and manipulate constraints.

````{admonition} Constraint Move: Output Correctness
:class: important

If the protocol is correct (its output is equal to the ideal functionality's output) then you can introduce a constraint saying that the two are equal. 

Why it's allowed: The protocol's output has already been shown to be equal to the ideal functionality's output.
````

````{admonition} Constraint Move: Constraint Introduction
:class: important

If the protocol contains the statement `x = e` then you can introduce the constraint `x = e`. 

Why it's allowed: The protocol enforces this constraint by explicitly computing it.
````

````{admonition} Constraint Move: Algebra
:class: important

You can perform algebra on constraints to derive new constraints.

Why it's allowed: Algebraic operations don't change the validity of the set of constraints.
````

#### Hybrid Moves

To actually build new hybrids in the sequence, we'll need to modify the program. We'll make these modifications using the following *hybrid moves*, valid rules of thumb for modifying a hybrid to build a new one without changing the joint distribution over the observations of the adversary.

````{admonition} Hybrid Move: Constraint Application
:class: important

If the protocol contains the statement `x = e1` where `e1` is some expression, and you have the constraint `e1 = e2` where `e2` is some expression, then you can replace the statement with `x = e2`. 

Why it's allowed: Constraints are assumed to be true, and they give you an alternative way to compute the same thing.
````

````{admonition} Hybrid Move: Unused Share
:class: important

If the protocol contains the statement `x1, x2 = share(x)` and `x1` is not used in the rest of the protocol, then you can replace the statement with `x2 = random()`.

Why it's allowed: A single secret share in isolation appears uniformly random and gives no information about the secret.
````

#### Example

Starting from `hyb1_sum` above, let's build the proof! First, we'll introduce some constraints. We annotate each new constraint with the constraint move that supports its validity.
1. `total` = `functionality_output` (output correctness)
2. `total` = `sum1 + sum2` (constraint introduction)
3. `sum1` = `x1 + y1` (constraint introduction)
4. `sum2` = `x2 + y2` (constraint introduction)
5. `sum2` = `functionality_output - sum1` (algebra on 1, 2, 3, 4)

Now, we can the last constraint to make hybrid moves and progress toward the simulator.

First, we use the **Constraint Application** move with constraint 5 to replace the definition of `sum2`:

In [6]:
def hyb2_sum(x, y, functionality_output):
    x1, x2 = share(x, 2).unlist(2)
    y1, y2 = share(y, 2).unlist(2)
    x2.send(p1, p2)
    y1.send(p2, p1)
    sum1 = x1 + y1
    sum2 = functionality_output - sum1
    sum1.send(p1, p2)
    sum2.send(p2, p1)
    total = reconstruct([sum1, sum2])
    return total

Next, we observe that `y2` is defined, but is never used in the program! So, we use the **Unused Share** move to eliminate `y2` from the program and replace `y1` with a random value. Once we've made this move, the honest party's input (`y`) is also no longer used, so we can eliminate it from the set of inputs to the program.

In [7]:
def hyb3_sum(x, functionality_output):
    x1, x2 = share(x, 2).unlist(2)
    y1 = p2.constant(GF.Random())
    x2.send(p1, p2)
    y1.send(p2, p1)
    sum1 = x1 + y1
    sum2 = functionality_output - sum1
    sum1.send(p1, p2)
    sum2.send(p2, p1)
    total = reconstruct([sum1, sum2])
    return total

This hybrid is our simulator! From the adversary's perspective, it's indistinguishable from the real protocol, given the output of the ideal functionality, due to the hybrid argument that `protocol_sum == hyb1_sum == hyb2_sum == hyb3_sum`. Since we're able to build this simulator, and it doesn't use P2's input at all, we can say that the protocol *securely realizes the ideal functionality* in the presence of a semi-honest adversary when P1 is corrupted and P2 is honest.

It's easy to see that a similar set of moves would let us build a sequence of hybrids for the case where P2 is corrupted and P1 is honest, since the protocol is essentially symmetric between the parties.